### IMSI allocation
- Помогает выбрать свободные IMSI / MSISDN / ICCID на основании данных RoamDb
- Проверяет выбранные IMSI / MSISDN по базам данных DMI/OSC
- Готовит объединенные post perso файлы

In [1]:
from collections import namedtuple
import pandas as pd
import roamability as rb
from pandas import DataFrame
from os.path import join

def print_check_result(df, db_name, table_name, imsi_range_start, imsi_range_end):
    print(f'Checking {db_name} {table_name} for: {imsi_range_start} - {imsi_range_end}', end='')
    if df.empty:
        print(' - No items found')
    else:
        display(df)

def check_dmi_imsi(imsi_range_start, imsi_range_end):
    sql_srt = '''
    SELECT SUBSTR(SI.IMSI_NUMBER,1,10) AS IMSI_RANGE, count(SI.IMSI_NUMBER) AS NUM
    FROM s_imsi si
    WHERE SI.IMSI_NUMBER BETWEEN {} AND {}
    GROUP BY SUBSTR(SI.IMSI_NUMBER,1,10)
    ORDER BY SUBSTR(SI.IMSI_NUMBER,1,10) desc
    '''.format(imsi_range_start, imsi_range_end)
    with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
        df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    print_check_result(df, 'DMI', 's_imsi', imsi_range_start, imsi_range_end)
    
    sql_srt = '''
    select SUBSTR(S.IMSI,1,10) AS IMSI_RANGE, count(S.IMSI) AS NUM
    from subscriber s
    where S.IMSI BETWEEN {} AND {}
    group by SUBSTR(S.IMSI,1,10)
    order by SUBSTR(S.IMSI,1,10) desc
    '''.format(imsi_range_start, imsi_range_end)
    with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
        df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    print_check_result(df, 'DMI', 'subscriber', imsi_range_start, imsi_range_end)
    
def check_dmi_msisdn(imsi_range_start, imsi_range_end, msisdn_range_len):
    sql_srt = '''
    SELECT SUBSTR(SI.cli_msisdn,1,{2}) AS MSISDN_RANGE,
    MIN(SI.cli_msisdn) AS MIN_MSISDN,
    MAX(SI.cli_msisdn) AS MAX_MSISDN,
    count(SI.cli_msisdn) AS NUM
    FROM s_imsi si
    WHERE SI.cli_msisdn BETWEEN {0} AND {1}
    GROUP BY SUBSTR(SI.cli_msisdn,1,{2})
    ORDER BY SUBSTR(SI.cli_msisdn,1,{2}) desc
    '''.format(imsi_range_start, imsi_range_end, msisdn_range_len)
    with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
        df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    print_check_result(df, 'DMI', 's_imsi', imsi_range_start, imsi_range_end)
    
    sql_srt = '''
    SELECT SUBSTR(S.msisdn,1,{2}) AS MSISDN_RANGE,
    MIN(S.msisdn) AS MIN_MSISDN,
    MAX(S.msisdn) AS MAX_MSISDN,
    count(S.msisdn) AS NUM
    FROM subscriber s
    WHERE S.msisdn BETWEEN {0} AND {1}
    GROUP BY SUBSTR(S.msisdn,1,{2})
    ORDER BY SUBSTR(S.msisdn,1,{2}) desc
    '''.format(imsi_range_start, imsi_range_end, msisdn_range_len)
    with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
        df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    print_check_result(df, 'DMI', 'subscriber', imsi_range_start, imsi_range_end)
    
def check_ocs_imsi(imsi_range_start, imsi_range_end):
    sql_srt="""
    SELECT SUBSTRING(mi.imsi, 1, 12) AS IMSI_RANGE, count(mi.imsi) AS NUM
    FROM multi_imsi mi
    WHERE mi.imsi BETWEEN '{}' AND '{}'
    AND mi.end_date IS NULL
    GROUP BY SUBSTRING(mi.imsi, 1, 12)
    ORDER BY SUBSTRING(mi.imsi, 1, 12) DESC
    """.format(imsi_range_start, imsi_range_end)
    with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
        df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    print_check_result(df, 'OCS', 'multi_imsi', imsi_range_start, imsi_range_end)
    
    sql_srt="""
    SELECT SUBSTRING(si.imsi, 1, 12) AS IMSI_RANGE, count(si.imsi) AS NUM
    FROM SUBSCRIBER_IMSIS si
    WHERE si.imsi BETWEEN '{}' AND '{}'
    AND si.end_date IS NULL
    GROUP BY SUBSTRING(si.imsi, 1, 12)
    ORDER BY SUBSTRING(si.imsi, 1, 12) DESC
    """.format(imsi_range_start, imsi_range_end)
    with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
        df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    print_check_result(df, 'OCS', 'SUBSCRIBER_IMSIS', imsi_range_start, imsi_range_end)
    
def check_ocs_msisdn(imsi_range_start, imsi_range_end, msisdn_range_len):
    sql_srt="""
    DECLARE @range_Length INT
    SET @range_Length = {2}
    SELECT SUBSTRING(PHONE_NUMBER,1, @range_Length) AS MSISDN_RANGE,
    MIN(PHONE_NUMBER) AS MIN_MSISDN,
    MAX(PHONE_NUMBER) AS MAX_MSISDN,
    count(*) AS NUM
    FROM SUBSCRIBER_PHONE_NUMBERS
    WHERE PHONE_NUMBER BETWEEN '{0}' AND '{1}'
    AND end_date IS NULL
    GROUP BY SUBSTRING(PHONE_NUMBER,1, @range_Length)
    ORDER BY SUBSTRING(PHONE_NUMBER,1, @range_Length) DESC
    """.format(imsi_range_start, imsi_range_end, msisdn_range_len)
    with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
        df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    print_check_result(df, 'OCS', 'SUBSCRIBER_PHONE_NUMBERS', imsi_range_start, imsi_range_end)
    
def check_dmi_imsi_detailed(imsi_range_start, imsi_range_end):
    sql_srt = '''
    SELECT
    SUBSTR(si.IMSI_NUMBER,1,12) AS S_IMSI_RANGE,
    sp.NAME AS PROFILE_NAME,
    MIN(si.IMSI_NUMBER) AS MIN_IMSI,MAX(si.IMSI_NUMBER) AS MAX_IMSI,
    COUNT(*) AS NUM
    FROM S_IMSI si, SPONSOR sp WHERE
    si.SPONSOR_REF = sp.RI
    AND si.IMSI_NUMBER BETWEEN {} AND {}
    GROUP BY SUBSTR(si.IMSI_NUMBER,1,12),sp.NAME
    ORDER BY SUBSTR(si.IMSI_NUMBER,1,12)
    '''.format(imsi_range_start, imsi_range_end)
    with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
        df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    print_check_result(df, 'DMI', 's_imsi', imsi_range_start, imsi_range_end)
    
    sql_srt = '''
    SELECT
    m.NAME AS MVNO_NAME,
    SUBSTR(IMSI,1,12) AS IMSI_RANGE,
    COUNT(*) AS NUM_SUBS,
    MIN(IMSI) AS MIN_IMSI,MAX(IMSI) AS MAX_IMSI
    FROM SUBSCRIBER s, MVNO m WHERE
    m.RI = s.MVNO_REF
    AND IMSI BETWEEN {} AND {}
    GROUP BY m.NAME, SUBSTR(IMSI,1,12)
    ORDER BY SUBSTR(IMSI,1,12)
    '''.format(imsi_range_start, imsi_range_end)
    with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
        df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    print_check_result(df, 'DMI', 'subscriber', imsi_range_start, imsi_range_end)

In [13]:
%%time

class DefinedSponsorRanges:
    
    def __init__(self, sponsor_name, imsi_range_esim, imsi_range_own_hlr, imsi_range_dmi_external_hlr, imsi_range_test,
                imsi_range_dent, msisdn_range, rdb_sponsor_name):
        self.sponsor_name = sponsor_name
        self.imsi_range_esim = imsi_range_esim
        self.imsi_range_own_hlr = imsi_range_own_hlr
        self.imsi_range_dmi_external_hlr = imsi_range_dmi_external_hlr
        self.imsi_range_test = imsi_range_test
        self.imsi_range_dent = imsi_range_dent
        self.msisdn_range = msisdn_range
        self.rdb_sponsor_name = rdb_sponsor_name

        
class SponsorRanges:

    def __init__(self, defined_sponsor_ranges, range_type):
        self.defined_sponsor_ranges = defined_sponsor_ranges
        self.range_type = range_type
        self.imsi_group_ranges = self.imsi_group_ranges()
        self.msisdn_group_ranges = self.msisdn_group_ranges()
    
    def define_range(self):
        if self.range_type == 'esim':
            return self.defined_sponsor_ranges.imsi_range_esim
        elif self.range_type == 'own_hlr':
            return self.defined_sponsor_ranges.imsi_range_own_hlr
        elif self.range_type == 'dmi_external_hlr':
            return self.defined_sponsor_ranges.imsi_range_dmi_external_hlr
        elif self.range_type == 'test':
            return self.defined_sponsor_ranges.imsi_range_test
        elif self.range_type == 'own_hlr_dent':
            return self.defined_sponsor_ranges.imsi_range_dent
        
    def fetch_imsi_data_from_rdb(self):
        
        if self.define_range():
            sql_srt="""
            SELECT
            o.Name,i.RangeStart,i.RangeEnd,i.Allocation
            FROM ImsiRange i
            LEFT JOIN [dbo].[Sponsors] s ON i.SponsorID = s.ID
            LEFT JOIN [dbo].[Operators] o ON o.ID = s.OperatorID
            WHERE o.Name = '{0}'
            AND i.RangeStart LIKE '{1}%'
            AND i.RangeEnd LIKE '{1}%';""".format(self.defined_sponsor_ranges.rdb_sponsor_name, self.define_range())
            with rb.MssqlConnect('172.19.11.59', 'RoamDb', 'roamdb', 'roamdbpasswd') as cnxn:
                df_rdb_imsi_range = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
            df_rdb_imsi_range.Name.replace(
                {self.defined_sponsor_ranges.rdb_sponsor_name:self.defined_sponsor_ranges.sponsor_name}, inplace=True)
            return df_rdb_imsi_range
    
    def fetch_msisdn_data_from_rdb(self):
        
        sql_srt="""
        SELECT
        o.Name,m.RangeStart,m.RangeEnd,m.Description
        FROM SponsorFakeMsisdnRanges m
        LEFT JOIN [dbo].[Sponsors] s ON m.SponsorID = s.ID
        LEFT JOIN [dbo].[Operators] o ON o.ID = s.OperatorID
        WHERE o.Name = '{0}'
        AND m.RangeStart LIKE '{1}%'
        AND m.RangeEnd LIKE '{1}%';""".format(self.defined_sponsor_ranges.rdb_sponsor_name,
                                                self.defined_sponsor_ranges.msisdn_range)
        with rb.MssqlConnect('172.19.11.59', 'RoamDb', 'roamdb', 'roamdbpasswd') as cnxn:
            df_rdb_msisdn_range = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
        df_rdb_msisdn_range.Name.replace(
            {self.defined_sponsor_ranges.rdb_sponsor_name:self.defined_sponsor_ranges.sponsor_name}, inplace=True)
        df_rdb_msisdn_range[['RangeStart','RangeEnd']] = df_rdb_msisdn_range[['RangeStart','RangeEnd']].astype('int64')
        return df_rdb_msisdn_range
    
    def imsi_group_ranges(self):
        
        df_rdb_imsi = DataFrame()
        df_rdb_imsi_range = self.fetch_imsi_data_from_rdb()
        for Name, RangeStart, RangeEnd, Allocation in df_rdb_imsi_range.values:
            df_rdb_imsi = df_rdb_imsi.append(DataFrame({'SPONSOR':Name,
                                  'COMMENT':Allocation}, pd.Index(range(RangeStart, RangeEnd), name='IMSI'))) 
        df_rdb_imsi['IMSI'] = df_rdb_imsi.index
        list_agg_imsi = [('_MIN','min'),('_MAX','max'),('_NUM','nunique')]
        list_agg_comment = [('','unique')]
        list_col = ['SPONSOR', df_rdb_imsi.IMSI // 10 ** imsi_agg_precision]
        df_rdb_imsi_group = df_rdb_imsi.groupby(list_col)['IMSI','COMMENT'].agg({'IMSI': list_agg_imsi,
                                                                                 'COMMENT':list_agg_comment}).reset_index()
        df_rdb_imsi_group.columns = [''.join(col) for col in df_rdb_imsi_group.columns]
        df_rdb_imsi_group = pd.concat([df_rdb_imsi_group, DataFrame(df_rdb_imsi_group.COMMENT.tolist())], axis=1)
        df_rdb_imsi_group.drop('COMMENT', axis=1, inplace=True)

        return df_rdb_imsi_group
    
    def msisdn_group_ranges(self):
        
        df_rdb_msisdn = DataFrame()
        df_rdb_msisdn_range = self.fetch_msisdn_data_from_rdb()
        for Name, RangeStart, RangeEnd, Description in df_rdb_msisdn_range.values:
            df_rdb_msisdn = df_rdb_msisdn.append(DataFrame({'SPONSOR':Name,
                                  'COMMENT':Description}, pd.Index(range(RangeStart, RangeEnd), name='MSISDN'))) 
        df_rdb_msisdn['MSISDN'] = df_rdb_msisdn.index
        list_agg_msisdn = [('_MIN','min'),('_MAX','max'),('_NUM','nunique')]
        list_agg_comment = [('','unique')]
        list_col = ['SPONSOR', df_rdb_msisdn.MSISDN // 10 ** msisdn_agg_precision]
        df_rdb_msisdn_group = df_rdb_msisdn.groupby(list_col)['MSISDN','COMMENT'].agg({'MSISDN': list_agg_msisdn,
                                                                                 'COMMENT':list_agg_comment}).reset_index()
        df_rdb_msisdn_group.columns = [''.join(col) for col in df_rdb_msisdn_group.columns]
        df_rdb_msisdn_group = pd.concat([df_rdb_msisdn_group, DataFrame(df_rdb_msisdn_group.COMMENT.tolist())], axis=1)
        df_rdb_msisdn_group.drop('COMMENT', axis=1, inplace=True)

        return df_rdb_msisdn_group

#######################################################################################
# Define variables here (Part 1/3)
#######################################################################################

imsi_agg_precision = 2 # Aggregation presision for IMSI rangaes 3 - 1000, 2 - 100, 1 - 10
msisdn_agg_precision = 2 # Aggregation presision for MSISDN rangaes

range_type = 'own_hlr_dent' # range_type: esim, own_hlr, own_hlr_dent, dmi_external_hlr, test

# sponsor_name: 'S1' / 'S2' / 'S3'...
                
s1_defined_sponsor_ranges = DefinedSponsorRanges(sponsor_name = 'S1',
                                                 imsi_range_esim = '425019614',
                                                 imsi_range_own_hlr = '425019613',
                                                 imsi_range_dmi_external_hlr = '42501962',
                                                 imsi_range_test = '42501961400',
                                                 imsi_range_dent = '4250196141',
                                                 msisdn_range = '972541',
                                                 rdb_sponsor_name = 'Partner')

s2_defined_sponsor_ranges = DefinedSponsorRanges(sponsor_name = 'S2',
                                                 imsi_range_esim = '260060145',
                                                 imsi_range_own_hlr = '26006014',
                                                 imsi_range_dmi_external_hlr = '26006014',
                                                 imsi_range_test = '26006014500',
                                                 imsi_range_dent = '2600601451',
                                                 msisdn_range = '487913',
                                                 rdb_sponsor_name = 'P4 Sp z o o')

s4_defined_sponsor_ranges = DefinedSponsorRanges(sponsor_name = 'S4',
                                                 imsi_range_esim = '4540302276',
                                                 imsi_range_own_hlr = '4540302276',
                                                 imsi_range_dmi_external_hlr = '4540302276',
                                                 imsi_range_test = '454030227309',
                                                 imsi_range_dent = '',
                                                 msisdn_range = '852975',
                                                 rdb_sponsor_name = 'H3G/Hutchinson')

s5_defined_sponsor_ranges = DefinedSponsorRanges(sponsor_name = 'S5',
                                                 imsi_range_esim = '515030192',
                                                 imsi_range_own_hlr = '515030191',
                                                 imsi_range_dmi_external_hlr = '515030191',
                                                 imsi_range_test = '5150301900000',
                                                 imsi_range_dent = '',
                                                 msisdn_range = '63918',
                                                 rdb_sponsor_name = 'Smart Communications Inc')

s6_defined_sponsor_ranges = DefinedSponsorRanges(sponsor_name = 'S6',
                                                 imsi_range_esim = '260036611',
                                                 imsi_range_own_hlr = '260036610',
                                                 imsi_range_dmi_external_hlr = '260036610',
                                                 imsi_range_test = '2600366100000',
                                                 imsi_range_dent = '',
                                                 msisdn_range = '485079',
                                                 rdb_sponsor_name = 'Orange Polska S A')

s8_defined_sponsor_ranges = DefinedSponsorRanges(sponsor_name = 'S8',
                                                 imsi_range_esim = '2345000264',
                                                 imsi_range_own_hlr = '2345000264',
                                                 imsi_range_dmi_external_hlr = '2345000264',
                                                 imsi_range_test = '',
                                                 imsi_range_dent = '',
                                                 msisdn_range = '44779771',
                                                 rdb_sponsor_name = 'JT Jersey Limited')

# range_type: esim, own_hlr, dmi_external_hlr, test

s1_sponsor_ranges = SponsorRanges(defined_sponsor_ranges = s1_defined_sponsor_ranges,
                                  range_type = range_type)

s2_sponsor_ranges = SponsorRanges(defined_sponsor_ranges = s2_defined_sponsor_ranges,
                                  range_type = range_type)

# s4_sponsor_ranges = SponsorRanges(defined_sponsor_ranges = s4_defined_sponsor_ranges,
#                                   range_type = range_type)

# s5_sponsor_ranges = SponsorRanges(defined_sponsor_ranges = s5_defined_sponsor_ranges,
#                                   range_type = range_type)

# s6_sponsor_ranges = SponsorRanges(defined_sponsor_ranges = s6_defined_sponsor_ranges,
#                                   range_type = range_type)

# s8_sponsor_ranges = SponsorRanges(defined_sponsor_ranges = s8_defined_sponsor_ranges,
#                                   range_type = range_type)


# sponsor_ranges = [s1_sponsor_ranges, s2_sponsor_ranges, s4_sponsor_ranges,
#                   s5_sponsor_ranges, s6_sponsor_ranges, s8_sponsor_ranges]

sponsor_ranges = [s1_sponsor_ranges, s2_sponsor_ranges]

#######################################################################################

#######################################################################################

Wall time: 10.3 s


In [3]:
# Get IMSI ranges

for sponsor_range in sponsor_ranges:
    display(sponsor_range.imsi_group_ranges.loc[sponsor_range.imsi_group_ranges[1].isna()].head(20))

,SPONSOR,IMSI,IMSI_MIN,IMSI_MAX,IMSI_NUM,0,1
154,S1,4250196141154,425019614115400,425019614115499,100,DENT 100K Reserv configured EPG RG,None
155,S1,4250196141155,425019614115500,425019614115599,100,DENT 100K Reserv configured EPG RG,None
156,S1,4250196141156,425019614115600,425019614115699,100,DENT 100K Reserv configured EPG RG,None
157,S1,4250196141157,425019614115700,425019614115799,100,DENT 100K Reserv configured EPG RG,None
158,S1,4250196141158,425019614115800,425019614115899,100,DENT 100K Reserv configured EPG RG,None
159,S1,4250196141159,425019614115900,425019614115999,100,DENT 100K Reserv configured EPG RG,None
210,S1,4250196141210,425019614121000,425019614121099,100,DENT 100K Reserv configured EPG RG,None
211,S1,4250196141211,425019614121100,425019614121199,100,DENT 100K Reserv configured EPG RG,None
212,S1,4250196141212,425019614121200,425019614121299,100,DENT 100K Reserv configured EPG RG,None
213,S1,4250196141213,425019614121300,425019614121399,100,DENT 100K Reserv configured EPG RG,None


,SPONSOR,IMSI,IMSI_MIN,IMSI_MAX,IMSI_NUM,0,1
154,S2,2600601451154,260060145115400,260060145115499,100,DENT 100K Reserv configured EPG RG,None
155,S2,2600601451155,260060145115500,260060145115599,100,DENT 100K Reserv configured EPG RG,None
156,S2,2600601451156,260060145115600,260060145115699,100,DENT 100K Reserv configured EPG RG,None
157,S2,2600601451157,260060145115700,260060145115799,100,DENT 100K Reserv configured EPG RG,None
158,S2,2600601451158,260060145115800,260060145115899,100,DENT 100K Reserv configured EPG RG,None
159,S2,2600601451159,260060145115900,260060145115999,100,DENT 100K Reserv configured EPG RG,None
210,S2,2600601451210,260060145121000,260060145121099,100,DENT 100K Reserv configured EPG RG,None
211,S2,2600601451211,260060145121100,260060145121199,100,DENT 100K Reserv configured EPG RG,None
212,S2,2600601451212,260060145121200,260060145121299,100,DENT 100K Reserv configured EPG RG,None
213,S2,2600601451213,260060145121300,260060145121399,100,DENT 100K Reserv configured EPG RG,None


In [4]:
# Get MSISDN ranges

for sponsor_range in sponsor_ranges:
    display(sponsor_range.msisdn_group_ranges.loc[sponsor_range.msisdn_group_ranges[1].isna()].head(20))

,SPONSOR,MSISDN,MSISDN_MIN,MSISDN_MAX,MSISDN_NUM,0,1,2,3
1623,S1,9725411623,972541162300,972541162399,100,Main New Range,None,None,None
1624,S1,9725411624,972541162400,972541162499,100,Main New Range,None,None,None
1625,S1,9725411625,972541162500,972541162599,100,Main New Range,None,None,None
1626,S1,9725411626,972541162600,972541162699,100,Main New Range,None,None,None
1627,S1,9725411627,972541162700,972541162799,100,Main New Range,None,None,None
1628,S1,9725411628,972541162800,972541162899,100,Main New Range,None,None,None
1629,S1,9725411629,972541162900,972541162999,100,Main New Range,None,None,None
2180,S1,9725412180,972541218000,972541218099,100,Main New Range,None,None,None
2181,S1,9725412181,972541218100,972541218199,100,Main New Range,None,None,None
2182,S1,9725412182,972541218200,972541218299,100,Main New Range,None,None,None


,SPONSOR,MSISDN,MSISDN_MIN,MSISDN_MAX,MSISDN_NUM,0,1
200,S2,4879130200,487913020000,487913020099,100,Main Range Ext1,None
201,S2,4879130201,487913020100,487913020199,100,Main Range Ext1,None
202,S2,4879130202,487913020200,487913020299,100,Main Range Ext1,None
203,S2,4879130203,487913020300,487913020399,100,Main Range Ext1,None
204,S2,4879130204,487913020400,487913020499,100,Main Range Ext1,None
205,S2,4879130205,487913020500,487913020599,100,Main Range Ext1,None
206,S2,4879130206,487913020600,487913020699,100,Main Range Ext1,None
207,S2,4879130207,487913020700,487913020799,100,Main Range Ext1,None
208,S2,4879130208,487913020800,487913020899,100,Main Range Ext1,None
209,S2,4879130209,487913020900,487913020999,100,Main Range Ext1,None


In [5]:
# Записать выбранные диапазоны IMSI

imsi_range_object = namedtuple('ImsiRange', 'sponsor imsi_range_start imsi_range_end msisdn_start msisdn_end')

### Set the IMSI ranges here ###

s1_imsi_range = imsi_range_object('S1',
                                  '425019614115400', '425019614115499', # IMSI
                                  '972541162300', '972541162399')       # MSISDN

s2_imsi_range = imsi_range_object('S2',
                                  '260060145115400', '260060145115499', # IMSI
                                  '487913020000', '487913020099')         # MSISDN

s4_imsi_range = imsi_range_object('S4',
                                  '', '', # IMSI
                                  '', '')         # MSISDN

s5_imsi_range = imsi_range_object('S5',
                                  '', '', # IMSI
                                  '', '')         # MSISDN

s6_imsi_range = imsi_range_object('S6',
                                  '', '', # IMSI
                                  '', '')         # MSISDN

s8_imsi_range = imsi_range_object('S8',
                                  '', '', # IMSI
                                  '', '')         # MSISDN

#################################

# imsi_ranges = [s1_imsi_range, s2_imsi_range, s4_imsi_range, s5_imsi_range, s6_imsi_range, s8_imsi_range] 

imsi_ranges = [s1_imsi_range, s2_imsi_range]

In [6]:
for imsi_range in imsi_ranges:
    print(f'''{imsi_range.sponsor}: IMSI: {imsi_range.imsi_range_start} - {imsi_range.imsi_range_end} \
({int(imsi_range.imsi_range_end)-int(imsi_range.imsi_range_start) + 1});
          MSISDN: {imsi_range.msisdn_start} - {imsi_range.msisdn_end} \
({int(imsi_range.msisdn_end)-int(imsi_range.msisdn_start) + 1})
          ''')

S1: IMSI: 425019614115400 - 425019614115499 (100);
          MSISDN: 972541162300 - 972541162399 (100)
          
S2: IMSI: 260060145115400 - 260060145115499 (100);
          MSISDN: 487913020000 - 487913020099 (100)
          


In [7]:
# Проверить IMSI в DMI

for imsi_range in imsi_ranges:
    if imsi_range.imsi_range_start:
        check_dmi_imsi(imsi_range.imsi_range_start, imsi_range.imsi_range_end)

Checking DMI s_imsi for: 425019614115400 - 425019614115499 - No items found
Checking DMI subscriber for: 425019614115400 - 425019614115499 - No items found
Checking DMI s_imsi for: 260060145115400 - 260060145115499 - No items found
Checking DMI subscriber for: 260060145115400 - 260060145115499 - No items found


In [8]:
# Проверить IMSI в OCS

for imsi_range in imsi_ranges:
    if imsi_range.imsi_range_start:
        check_ocs_imsi(imsi_range.imsi_range_start, imsi_range.imsi_range_end)

Checking OCS multi_imsi for: 425019614115400 - 425019614115499 - No items found
Checking OCS SUBSCRIBER_IMSIS for: 425019614115400 - 425019614115499 - No items found
Checking OCS multi_imsi for: 260060145115400 - 260060145115499 - No items found
Checking OCS SUBSCRIBER_IMSIS for: 260060145115400 - 260060145115499 - No items found


In [9]:
# Проверить MSISDN в DMI

for imsi_range in imsi_ranges:
    check_dmi_msisdn(imsi_range.msisdn_start, imsi_range.msisdn_end, len(imsi_range.msisdn_start) - msisdn_agg_precision)  

Checking DMI s_imsi for: 972541162300 - 972541162399 - No items found
Checking DMI subscriber for: 972541162300 - 972541162399 - No items found
Checking DMI s_imsi for: 487913020000 - 487913020099 - No items found
Checking DMI subscriber for: 487913020000 - 487913020099 - No items found


In [10]:
# Проверить MSISDN в OCS

for imsi_range in imsi_ranges:
    check_ocs_msisdn(imsi_range.msisdn_start, imsi_range.msisdn_end, len(imsi_range.msisdn_start) - msisdn_agg_precision) 

Checking OCS SUBSCRIBER_PHONE_NUMBERS for: 972541162300 - 972541162399 - No items found
Checking OCS SUBSCRIBER_PHONE_NUMBERS for: 487913020000 - 487913020099 - No items found


In [79]:
# Детализированная информация по IMSI DMI

for imsi_range in imsi_ranges:
    if imsi_range.imsi_range_start:
        check_dmi_imsi_detailed(imsi_range.imsi_range_start, imsi_range.imsi_range_end)

Checking DMI s_imsi for: 425019613090000 - 425019613094999 - No items found
Checking DMI subscriber for: 425019613090000 - 425019613094999 - No items found
Checking DMI s_imsi for: 260060143150000 - 260060143154999 - No items found
Checking DMI subscriber for: 260060143150000 - 260060143154999 - No items found
Checking DMI s_imsi for: 454030227610000 - 454030227614999 - No items found
Checking DMI subscriber for: 454030227610000 - 454030227614999 - No items found
Checking DMI s_imsi for: 515030191110000 - 515030191114999 - No items found
Checking DMI subscriber for: 515030191110000 - 515030191114999 - No items found
Checking DMI s_imsi for: 260036610001000 - 260036610005999 - No items found
Checking DMI subscriber for: 260036610001000 - 260036610005999 - No items found
Checking DMI s_imsi for: 234500026420000 - 234500026424999 - No items found
Checking DMI subscriber for: 234500026420000 - 234500026424999 - No items found


In [14]:
# Проверить запись IMSI в RoamDb

for sponsor_range in sponsor_ranges:
    for imsi_range in imsi_ranges:
        if imsi_range.sponsor == sponsor_range.defined_sponsor_ranges.sponsor_name:
            display(sponsor_range.imsi_group_ranges.loc[
                (sponsor_range.imsi_group_ranges.IMSI_MIN >= int(imsi_range.imsi_range_start)) &
                (sponsor_range.imsi_group_ranges.IMSI_MIN <= int(imsi_range.imsi_range_end))].head(20))

,SPONSOR,IMSI,IMSI_MIN,IMSI_MAX,IMSI_NUM,0,1
154,S1,4250196141154,425019614115400,425019614115499,100,DENT 100K Reserv configured EPG RG,Dent_200520_100


,SPONSOR,IMSI,IMSI_MIN,IMSI_MAX,IMSI_NUM,0,1
154,S2,2600601451154,260060145115400,260060145115499,100,DENT 100K Reserv configured EPG RG,Dent_200520_100


### ICCID allocation

In [17]:
sql_srt="""
SELECT 
o.Name,siccid.RangeStart,siccid.RangeEnd,siccid.Allocation
FROM [dbo].[SponsorICCID] siccid
LEFT JOIN [dbo].[Sponsors] s ON siccid.SponsorID = s.ID
LEFT JOIN [dbo].[Operators] o ON o.ID = s.OperatorID
WHERE o.Name LIKE 'X2%';"""
with rb.MssqlConnect('172.19.11.59', 'RoamDb', 'roamdb', 'roamdbpasswd') as cnxn:
    df_iccid = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_iccid.head()

df_iccid['Count'] = df_iccid.RangeEnd - df_iccid.RangeStart + 1
dict_agg = {'RangeStart':'min', 'RangeEnd':'max', 'Count':'sum'}
list_col = ['Name', 'Allocation', df_iccid.RangeStart.astype('str').str.slice(0,12)]
df_iccid.groupby(list_col, as_index=False).agg(dict_agg).sort_values(by=['RangeStart','RangeEnd'])

,Name,Allocation,RangeStart,RangeEnd,Count
1,X2one,ALL 190826,8997212330000000000,8997212330099999999,100000000
6,X2one,Dailoq 500 190830,8997212330099000000,8997212330099000499,500
0,X2one,1K DIALOQ #3501982,8997212330099000500,8997212330099001499,1000
19,X2one,Reserved for DENT 8.5K,8997212330099001500,8997212330099009999,8500
13,X2one,Dialoq 5 test eSim,8997212330099010000,8997212330099010004,5
16,X2one,For Tests,8997212330099010000,8997212330099010999,1000
2,X2one,DENT 191122,8997212330099010005,8997212330099010206,202
3,X2one,DENT 191127,8997212330099010207,8997212330099010706,500
21,X2one,Tcom Test 200104,8997212330099010707,8997212330099010716,10
4,X2one,DENT 200114,8997212330099010717,8997212330099011216,500


In [16]:
the_last = 8997212330099114709
num = 100
print(f'{the_last + 1}-{the_last + num}')
print(f"BETWEEN '{the_last + 1}' AND '{the_last + num}'")

8997212330099114710-8997212330099114809
BETWEEN '8997212330099114710' AND '8997212330099114809'


### Join post perso file with IMSI and split to test and prod batches

In [16]:
downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA\PostPerso'

customer_ranges_object = namedtuple('CustomerRanges',
                                    'input_file_keys input_files_list imsi_start imsi_end iccid_start iccid_end')

#######################################################################################
# Define variables here
#######################################################################################

customer_range = customer_ranges_object('Flexi_200402_20K.txt', # File with keys
                                        ['Flexi_200402_20K_Perso_PROF1.txt', # Profiles
                                         'Flexi_200402_20K_Perso_PROF2.txt',
                                         'Flexi_200402_20K_Perso_PROF3.txt'],
                                        425019613095000, 425019613114999, # IMSI
                                        8997212330099074610, 8997212330099094609) # ICCID

test_batch_length = 10

#######################################################################################

In [20]:
# Prepare test and prod input files for subscribers in RB HLR

# The first input file (input_file_keys)
# IMSI KI OPC
# 425019613095010 200799CA5F46B0B19F55300B18B36149 F0F0213DA789466AEC5C2C16B2722EF7

# The input files withICCID and profile string without headers (_Perso_PROFX)
# 8935913000000700000 D15602028381060281000B4C4406812143...

for input_file in customer_range.input_files_list:
    imsi_range = range(customer_range.imsi_start, customer_range.imsi_end+1)
    iccid_range = range(customer_range.iccid_start, customer_range.iccid_end+1)
    if len(iccid_range) != len(imsi_range):
        print(f'ICCID range ({len(iccid_range)}) is not equal to IMSI range ({len(imsi_range)})')
        break
    else:
        
        df_keys = pd.read_csv(join(downloads, customer_range.input_file_keys), sep=' ', dtype='str')
        df2 = DataFrame({'ICCID': range(customer_range.iccid_start, customer_range.iccid_end+1),
                         'IMSI': range(customer_range.imsi_start, customer_range.imsi_end+1)}, dtype='str')
        df3 = pd.read_csv(join(downloads, input_file), sep=' ', names=['ICCID', 'PROFILEDATA'], dtype='str')
        df = pd.merge(df2, df_keys, how='outer', on='IMSI')
        df = pd.merge(df, df3, how='outer', on='ICCID')
        display(df)
    if len(iccid_range) == len(imsi_range) == len(df):
        print(f'The length of the ranges is correct and equal to {len(df)}\n')
        output_file_test = input_file[:-4] + '_test.csv'
        output_file_prod = input_file[:-4] + '_prod.csv'
        df[:test_batch_length].to_csv(join(downloads, output_file_test), index=False)
        df[test_batch_length:].to_csv(join(downloads, output_file_prod), index=False)
        print(f"""The input file {input_file} was split into:
        Test file: {output_file_test}
        Prod file: {output_file_prod}""")
    else:
        print(f'The length of the ranges is incorrect:\n ICCID={len(iccid_range)}\nIMSI={len(imsi_range)}\nDF={len(df)}')

,ICCID,IMSI,KI,OPC,PROFILEDATA
0,8997212330099074610,425019613095000,42099172E9D6507549E5BCD05B921E25,65FB7630B065BCAFFD42FF7A39E5C8A1,D15602028381060281000B4C4406812143657FF6115030...
1,8997212330099074611,425019613095001,E177792BD48B45D948D9BDA7A3C7EB27,9A2160DAD109A1FB0C96F138A951EEBB,D15602028381060281000B4C4406812143657FF6115030...
2,8997212330099074612,425019613095002,608AFD25D4699ED10089C7D27B62DBA1,EB86E5E8D89FF4EE51FC2122279D1F53,D15602028381060281000B4C4406812143657FF6115030...
3,8997212330099074613,425019613095003,6A77A2E341478C1E119DBFEF9F1DB778,6FDC73F5614ADDC151BB68BD74F9958B,D15602028381060281000B4C4406812143657FF6115030...
4,8997212330099074614,425019613095004,29488B931C5871E417EAABF4AAEECF35,898B6F5708DE48C30C66C71CCF8E48B7,D15602028381060281000B4C4406812143657FF6115030...
...,...,...,...,...,...
19995,8997212330099094605,425019613114995,01752E6802EA355DFC74FFE8B8F5D5D6,322C5FAD874B28B944FC049AE9224D40,D15602028381060281000B4C4406812143657FF6115030...
19996,8997212330099094606,425019613114996,820D64F784C35B0BEA486A79A71C5E88,5AB149FBD3603435B5CCA3A16864C500,D15602028381060281000B4C4406812143657FF6115030...
19997,8997212330099094607,425019613114997,4EF3001F33857D9D4C62CA47CC7412E3,E67BB34F8A6929A8A0D92C2BE14A36F7,D15602028381060281000B4C4406812143657FF6115030...
19998,8997212330099094608,425019613114998,ECE6334570BCA18348948D6A8324DE6F,C5862DCF2193C5E1AAD09AF2EC3D5798,D15602028381060281000B4C4406812143657FF6115030...


The length of the ranges is correct and equal to 20000

The input file Flexi_200402_20K_Perso_PROF1.txt was split into:
        Test file: Flexi_200402_20K_Perso_PROF1_test.csv
        Prod file: Flexi_200402_20K_Perso_PROF1_prod.csv


,ICCID,IMSI,KI,OPC,PROFILEDATA
0,8997212330099074610,425019613095000,42099172E9D6507549E5BCD05B921E25,65FB7630B065BCAFFD42FF7A39E5C8A1,D15602028381060281000B4C4406812143657FF6115030...
1,8997212330099074611,425019613095001,E177792BD48B45D948D9BDA7A3C7EB27,9A2160DAD109A1FB0C96F138A951EEBB,D15602028381060281000B4C4406812143657FF6115030...
2,8997212330099074612,425019613095002,608AFD25D4699ED10089C7D27B62DBA1,EB86E5E8D89FF4EE51FC2122279D1F53,D15602028381060281000B4C4406812143657FF6115030...
3,8997212330099074613,425019613095003,6A77A2E341478C1E119DBFEF9F1DB778,6FDC73F5614ADDC151BB68BD74F9958B,D15602028381060281000B4C4406812143657FF6115030...
4,8997212330099074614,425019613095004,29488B931C5871E417EAABF4AAEECF35,898B6F5708DE48C30C66C71CCF8E48B7,D15602028381060281000B4C4406812143657FF6115030...
...,...,...,...,...,...
19995,8997212330099094605,425019613114995,01752E6802EA355DFC74FFE8B8F5D5D6,322C5FAD874B28B944FC049AE9224D40,D15602028381060281000B4C4406812143657FF6115030...
19996,8997212330099094606,425019613114996,820D64F784C35B0BEA486A79A71C5E88,5AB149FBD3603435B5CCA3A16864C500,D15602028381060281000B4C4406812143657FF6115030...
19997,8997212330099094607,425019613114997,4EF3001F33857D9D4C62CA47CC7412E3,E67BB34F8A6929A8A0D92C2BE14A36F7,D15602028381060281000B4C4406812143657FF6115030...
19998,8997212330099094608,425019613114998,ECE6334570BCA18348948D6A8324DE6F,C5862DCF2193C5E1AAD09AF2EC3D5798,D15602028381060281000B4C4406812143657FF6115030...


The length of the ranges is correct and equal to 20000

The input file Flexi_200402_20K_Perso_PROF2.txt was split into:
        Test file: Flexi_200402_20K_Perso_PROF2_test.csv
        Prod file: Flexi_200402_20K_Perso_PROF2_prod.csv


,ICCID,IMSI,KI,OPC,PROFILEDATA
0,8997212330099074610,425019613095000,42099172E9D6507549E5BCD05B921E25,65FB7630B065BCAFFD42FF7A39E5C8A1,D15602028381060281000B4C4406812143657FF6115030...
1,8997212330099074611,425019613095001,E177792BD48B45D948D9BDA7A3C7EB27,9A2160DAD109A1FB0C96F138A951EEBB,D15602028381060281000B4C4406812143657FF6115030...
2,8997212330099074612,425019613095002,608AFD25D4699ED10089C7D27B62DBA1,EB86E5E8D89FF4EE51FC2122279D1F53,D15602028381060281000B4C4406812143657FF6115030...
3,8997212330099074613,425019613095003,6A77A2E341478C1E119DBFEF9F1DB778,6FDC73F5614ADDC151BB68BD74F9958B,D15602028381060281000B4C4406812143657FF6115030...
4,8997212330099074614,425019613095004,29488B931C5871E417EAABF4AAEECF35,898B6F5708DE48C30C66C71CCF8E48B7,D15602028381060281000B4C4406812143657FF6115030...
...,...,...,...,...,...
19995,8997212330099094605,425019613114995,01752E6802EA355DFC74FFE8B8F5D5D6,322C5FAD874B28B944FC049AE9224D40,D15602028381060281000B4C4406812143657FF6115030...
19996,8997212330099094606,425019613114996,820D64F784C35B0BEA486A79A71C5E88,5AB149FBD3603435B5CCA3A16864C500,D15602028381060281000B4C4406812143657FF6115030...
19997,8997212330099094607,425019613114997,4EF3001F33857D9D4C62CA47CC7412E3,E67BB34F8A6929A8A0D92C2BE14A36F7,D15602028381060281000B4C4406812143657FF6115030...
19998,8997212330099094608,425019613114998,ECE6334570BCA18348948D6A8324DE6F,C5862DCF2193C5E1AAD09AF2EC3D5798,D15602028381060281000B4C4406812143657FF6115030...


The length of the ranges is correct and equal to 20000

The input file Flexi_200402_20K_Perso_PROF3.txt was split into:
        Test file: Flexi_200402_20K_Perso_PROF3_test.csv
        Prod file: Flexi_200402_20K_Perso_PROF3_prod.csv


In [8]:
# Prepare test and prod input files if the subscriber is in Customer's HLR

# The input file format is with ICCID and profile string without headers
# 8935913000000700000 D15602028381060281000B4C4406812143...

imsi_range = range(customer_range.imsi_start, customer_range.imsi_end+1)
iccid_range = range(customer_range.iccid_start, customer_range.iccid_end+1)

df = DataFrame({'ICCID': range(customer_range.iccid_start, customer_range.iccid_end+1),
                 'IMSI': range(customer_range.imsi_start, customer_range.imsi_end+1)}, dtype='str')

for i, input_file in enumerate(customer_range.input_files_list):
    if len(iccid_range) != len(imsi_range):
        print(f'ICCID range ({len(iccid_range)}) is not equal to IMSI range ({len(imsi_range)})')
        break
    else:
        df1 = pd.read_csv(join(downloads, input_file), sep=' ', names=['ICCID', f'PROFILEDATA{i+1}'], dtype='str')
        df = pd.merge(df, df1, how='outer', on='ICCID')

display(df.head(3))

if len(iccid_range) == len(imsi_range) == len(df):
    print(f'The length of the ranges is correct and equal to {len(df)}\n')
    output_file_test = input_file[:-10] + '_test.csv'
    output_file_prod = input_file[:-10] + '_prod.csv'
    df[:test_batch_length].to_csv(join(downloads, output_file_test), index=False)
    df[test_batch_length:].to_csv(join(downloads, output_file_prod), index=False)
    print(f"""The input file {input_file} was split into:
    Test file: {output_file_test}
    Prod file: {output_file_prod}""")
else:
    print(f'The length of the ranges is incorrect:\n ICCID={len(iccid_range)}\nIMSI={len(imsi_range)}\nDF={len(df)}')

FileNotFoundError: [Errno 2] File b'c:\\Users\\balob\\Documents\\GITLAB\\RB_BD\\DATA\\PostPerso\\Tcom_200225_5K_Perso_PROF1.txt' does not exist: b'c:\\Users\\balob\\Documents\\GITLAB\\RB_BD\\DATA\\PostPerso\\Tcom_200225_5K_Perso_PROF1.txt'

### Developments